## Imports

In [2]:
# Note this script needs a conda environment with sbibm installed, e.g.:

# using Conda
# using Pkg

# ENV["PYTHON"] = ""
# Pkg.build("PyCall")

# Conda.pip_interop(true)
# Conda.pip("install", "sbibm")

# Make sure up to date
# Pkg.rm("SyntheticLikelihood")
# Pkg.add(url="https://github.com/danielward27/SyntheticLikelihood.jl")
using SyntheticLikelihood
using PyCall
using Distributions
using DelimitedFiles
using Random
using Parameters
using LinearAlgebra

sbibm = pyimport("sbibm")
torch = pyimport("torch");

## Convert stuff from python to julia

In [21]:
task_priors = include("task_priors.jl")
String.(keys(task_priors))

("gaussian_linear", "gaussian_linear_uniform", "gaussian_mixture", "sir", "bernoulli_glm")

### Rough test that prior conversion looks right

In [5]:
for task_name in String.(keys(task_priors))
    n = 2000
    jl_prior = task_priors[Symbol(task_name)]
    jl_samples = sample_θ(jl_prior, n)
    jl_mean = mean.(eachcol(jl_samples))
    jl_cov = cov(jl_samples)

    py_prior = sbibm.get_task(task_name).get_prior()
    py_samples = py_prior(n).numpy();
    py_mean = mean.(eachcol(py_samples))
    py_cov = cov(py_samples)
    
    println(task_name)
    println("Julia means = $(round.(jl_mean; digits = 2))")
    println("Python means = $(round.(py_mean; digits = 2)) \n")
    
    @assert size(py_mean) == size(jl_mean)
    @assert isapprox(py_mean, jl_mean; rtol = 2)
    @assert isapprox(py_cov, jl_cov; rtol = 0.7)
end

gaussian_linear
Julia means = [0.0, 0.0, 0.01, 0.01, 0.01, 0.0, -0.01, -0.0, 0.0, -0.01]
Python means = Float32[-0.01, -0.0, 0.02, 0.0, -0.0, -0.0, 0.0, 0.0, 0.01, -0.01] 

gaussian_linear_uniform
Julia means = [0.01, -0.01, 0.0, -0.0, 0.01, 0.01, -0.0, 0.0, 0.0, 0.02]
Python means = Float32[-0.01, -0.01, -0.0, 0.01, 0.01, 0.0, 0.01, 0.01, -0.02, -0.01] 

gaussian_mixture
Julia means = [-0.04, -0.04]
Python means = Float32[-0.03, -0.18] 

sir
Julia means = [0.45, 0.13]
Python means = Float32[0.45, 0.13] 

bernoulli_glm
Julia means = [-0.04, 0.01, -0.0, 0.0, 0.02, 0.03, 0.01, -0.0, -0.03, -0.02]
Python means = Float32[0.0, 0.03, 0.07, 0.06, 0.02, -0.01, -0.02, -0.03, -0.02, 0.0] 

bernoulli_glm_raw
Julia means = [0.03, 0.02, 0.04, 0.04, 0.05, 0.05, 0.02, -0.0, -0.02, -0.01]
Python means = Float32[-0.01, -0.02, 0.01, -0.0, -0.02, -0.02, -0.02, -0.01, -0.02, -0.02] 



In [6]:
function get_jl_simulator(task)   
    py_simulator = task.get_simulator()
    simulator(θ::Vector{Float64}) = begin
        θ = torch.tensor(θ, dtype = torch.float32)
        x = py_simulator(θ)
        convert(Vector{Float64}, vec(x.numpy()))
    end
    simulator
end;

In [7]:
struct JuliaTask
    name
    simulator
    prior
    s_true
    obs_seed
end

function JuliaTask(python_task, obs_seed::Integer)
    name = python_task.name
    simulator = get_jl_simulator(python_task)
    prior = task_priors[Symbol(name)]
    s_true = vec(python_task.get_observation(obs_seed).numpy())
    s_true = convert(Vector{Float64}, s_true)
    JuliaTask(name, simulator, prior, s_true, obs_seed)
end;

## Loop through tasks and run the Riemannian ULA algorithm

In [ ]:
ADD IN RUN TIMES BELOW!

In [9]:
for (i, task_name) in enumerate(String.(keys(task_priors)))
    @info "Task = $(task_name)"

    Random.seed!(i)
    pytask = sbibm.get_task(task_name)
    jltask = JuliaTask(pytask, 1)
    
    @unpack simulator, prior, s_true, obs_seed = jltask
    
    n_steps = 4000
    init_θ = sample_θ(prior)

    local_posterior = LocalPosterior(;
      simulator,
      s_true,
      n_sim = 1000,
      prior,
    )
    
    rula = RiemannianULA(0.2)
    
    try
        data = run_sampler!(rula, local_posterior; init_θ, n_steps)
        open("./samples/$(task_name)_rula.txt", "w") do io
            writedlm(io, data.θ)
        end
    catch e
        @warn "$(task_name) failed!"
    end
end

┌ Info: Task = gaussian_linear
└ @ Main In[9]:2
Progress: 100%|█████████████████████████████████████████| Time: 0:24:19
┌ Info: Task = gaussian_linear_uniform
└ @ Main In[9]:2
Progress: 100%|█████████████████████████████████████████| Time: 0:25:39
┌ Info: Task = gaussian_mixture
└ @ Main In[9]:2
Progress: 100%|█████████████████████████████████████████| Time: 0:12:03
┌ Info: Task = sir
└ @ Main In[9]:2
Progress:   2%|█                                        |  ETA: 2:23:19┌ Warning: GLM did not converge. Corresponding variance set to sample
│ covariance.
└ @ SyntheticLikelihood /home/dw16200/.julia/packages/SyntheticLikelihood/NjVDE/src/glm_local_regression.jl:69
Progress:   5%|██                                       |  ETA: 2:18:34┌ Warning: GLM did not converge. Corresponding variance set to sample
│ covariance.
└ @ SyntheticLikelihood /home/dw16200/.julia/packages/SyntheticLikelihood/NjVDE/src/glm_local_regression.jl:69
Progress:  15%|██████▎                                  |  ETA:

## Loop through tasks and run basic Bayesian Synthetic Likelihood
Below we use standard synthetic likelihood. We use a burn in of 1000 samples, and then use the empirical covariance matrix of the last 500 samples of the burn in to form the proposal distribution.

In [ ]:
basic_posterior = BasicPosterior(; simulator, s_true, prior)
rwm = RWMetropolis(MvNormal(cov(expected)))

data = run_sampler!(rwm, basic_posterior; init_θ, n_steps = 4000,
  collect_data = [:θ, :accepted])

θ = data.θ
@test isapprox(mean(expected), mean.(eachcol(θ)); atol = 2)
acceptance_rate = data.accepted[end]/1000

# hcat(unique.(eachcol(θ))...)
110/1000

using Plots
plot(θ)

In [ ]:
#using DelimitedFiles
#using GLM
#X = readdlm("X.txt")
#y = readdlm("y.txt")
#y = reshape(y, length(y))
#glm(X, y, Gamma(), LogLink(), maxiter=1000)